In [1]:
import pickle
import tflearn
import tensorflow as tf
import random
import numpy as np
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

W0822 22:46:48.835273 140737086989248 deprecation_wrapper.py:119] From /anaconda3/envs/chatbot_env/lib/python3.6/site-packages/tflearn/helpers/summarizer.py:9: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.

W0822 22:46:48.838706 140737086989248 deprecation_wrapper.py:119] From /anaconda3/envs/chatbot_env/lib/python3.6/site-packages/tflearn/helpers/trainer.py:25: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W0822 22:46:48.867875 140737086989248 deprecation_wrapper.py:119] From /anaconda3/envs/chatbot_env/lib/python3.6/site-packages/tflearn/collections.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0822 22:46:48.883775 140737086989248 deprecation_wrapper.py:119] From /anaconda3/envs/chatbot_env/lib/python3.6/site-packages/tflearn/config.py:123: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.

W0822 22:46:48.898

In [2]:
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']
ERROR_THRESHOLD = 0.10
context = {}
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [16]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=60, batch_size=2, show_metric=True)
model.save('model.tflearn')

Training Step: 1439  | total loss: 0.50847 | time: 0.078s
| Adam | epoch: 060 | loss: 0.50847 - acc: 0.9492 -- iter: 46/47
Training Step: 1440  | total loss: 0.49987 | time: 0.081s
| Adam | epoch: 060 | loss: 0.49987 - acc: 0.9542 -- iter: 47/47
--


In [17]:
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]

    #print('RESULTS:', results)
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    #print('RESULTS pruned:', results)
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    print('Return list:', return_list)
    return return_list

In [18]:
def response(sentence, userID='123', show_details=False):
    results = classify(sentence)

    #print(context)
    if results:
       
        while results:
            #print('Results[][]:',results[0][0])
            for i in intents['intents']:

                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        #print(context)
                        return random.choice(i['responses'])

            results.pop(0)

In [19]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [20]:
def bow(sentence, words, show_details=False):

    sentence_words = clean_up_sentence(sentence)

    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [21]:
import sys
print('Welcome to Car Rental Service')



Welcome to Car Rental Service


In [24]:

from time import sleep
while True:
    query = input('Customer: ')
    resp = response(query)
    print('typing...')
    sleep(1)

    print(resp)

Customer: hello
Return list: [('greeting', 0.3674209), ('services', 0.32942325), ('hours', 0.14991069), ('thanks', 0.10513929)]
typing...
Hello, thanks for visiting XYZ rental car service
Customer: hi
Return list: [('greeting', 0.37028167), ('services', 0.3279913), ('hours', 0.15594815)]
typing...
Hi there, how can I help?
Customer: i need to rent a car
Return list: [('rental', 0.9255801)]
typing...
Great! We have cars available. Are you looking to rent today or later this week?
Customer: today
Return list: [('rentaltoday', 0.4617855), ('cars', 0.3149021), ('rental', 0.15222153)]
typing...
For rentals today please call +92-321-XYZCARS


KeyboardInterrupt: 